In [19]:
import pandas as pd

import osmosys.osmosys
import activityinfo
from activityinfo import Client
from activityinfo import FormProcessing
import importlib
import model.modelAI
from activityinfo.id import generate_id
import json
from requests.exceptions import HTTPError
import os.path
import numpy as np

importlib.reload(osmosys.osmosys)
importlib.reload(activityinfo.FormProcessing)
importlib.reload(model.modelAI)

<module 'model.modelAI' from 'D:\\proyectos\\OSMOSYS_2023\\IntegracionAI2023\\AI_OsmosysIntegrationScripts2023\\model\\modelAI.py'>

In [20]:
## parameters
indicatorCodeAI = 'IN1_01'
month = 'ENERO'
month_number = '2023-01'
year = 2023
test = True
print(
    '--------------------------------------' + indicatorCodeAI + '------------------------------------------------------')


--------------------------------------IN1_01------------------------------------------------------


In [21]:
# busco matchs
subformDf = osmosys.osmosys.getMatchSubforms(indicatorCodeAI)
## print(subformDf.info)
indicatorIdsOsmosys = subformDf.osmosys_indicator_id.unique()
subformSeries = subformDf.iloc[0]
formIdAI = subformSeries.form_id
parentFormIdAI = subformSeries.parent_form_id
indicatorIdAI = subformSeries.indicador_ai_id
print("formIdAI: " + formIdAI)
print("parentFormIdAI: " + parentFormIdAI)
print("indicatorIdAI: " + indicatorIdAI)
print("indicatorIdsOsmosys: " + str(indicatorIdsOsmosys))

formIdAI: c7xnpifldnik44oign
parentFormIdAI: cg7ke70ldnik44oih1
indicatorIdAI: cx66m8dlcza9pfs3x
indicatorIdsOsmosys: [265 267 270 276 278]


In [26]:

## consulto los datos reportados
print("indicatorIdsOsmosys: " + str(indicatorIdsOsmosys))
indicatorIdsOsmosysTranferencias = np.array([267, 270])
print("indicatorIdsOsmosysTranferencias: " + str(indicatorIdsOsmosysTranferencias))
indicatorIdsOsmosysServicios = np.setdiff1d(indicatorIdsOsmosys, indicatorIdsOsmosysTranferencias)
print("indicatorIdsOsmosysServicios: " + str(indicatorIdsOsmosysServicios))

reportedDfServicios = osmosys.osmosys.getIesPartnerCantonsByIndicatorsIdsOsmosysAndMonth(
    indicatorsIdsOmosys=indicatorIdsOsmosysServicios,
    month=month, year=year)
reportedDfTransferencias = osmosys.osmosys.getIesPartnerCantonsByIndicatorsIdsOsmosysAndMonth(
    indicatorsIdsOmosys=indicatorIdsOsmosysTranferencias,
    month=month, year=year)
print('reported data Servicios: ' + str(reportedDfServicios.shape[0]))
print('reported data transferencias: ' + str(reportedDfTransferencias.shape[0]))

indicatorIdsOsmosys: [265 267 270 276 278]
indicatorIdsOsmosysTranferencias: [267 270]
indicatorIdsOsmosysServicios: [265 276 278]
reported data Servicios: 2
reported data transferencias: 1


In [27]:
## obtengo los parents de AI
parentFormsIdsServicios = FormProcessing.getParentsIds(parentFormIdAI=parentFormIdAI, reportedDf=reportedDfServicios)
parentFormsIdsTransferencias = FormProcessing.getParentsIds(parentFormIdAI=parentFormIdAI,
                                                            reportedDf=reportedDfTransferencias)

In [34]:
## contruyo las estructuras de datos
changesList = []
newIds = []
## los que  son servicios
for index, row in parentFormsIdsServicios.iterrows():
    # print(row)
    orgAcron = row.acronym
    cantonCode = row.canton_code
    df = osmosys.osmosys.getRefValues(year=year, month=month, orgOsmosys=orgAcron,
                                      indicatorsIdsOmosys=indicatorIdsOsmosysServicios, cantonCode=cantonCode)

    poblacion_meta = ["Refugiados/as y migrantes", "Comunidad de acogida"]

    IN1_01_RM_MM_N = int(df.loc[df['age_gender'] == 'ADULTAS'].iloc[0].value_a)
    IN1_01_RM_HH_N = int(df.loc[df['age_gender'] == 'ADULTOS'].iloc[0].value_a)
    IN1_01_RM_OTR_N = int(df.loc[df['age_gender'] == 'OTRO'].iloc[0].value_a)

    dfCa = osmosys.osmosys.getCAValues(year=year, month=month, orgOsmosys=orgAcron,
                                       indicatorsIdsOmosys=indicatorIdsOsmosysServicios, cantonCode=cantonCode)

    IN1_01_CA_MM_N = int(dfCa.loc[df['age_gender'] == 'ADULTAS'].iloc[0].value_a)
    IN1_01_CA_HH_N = int(dfCa.loc[df['age_gender'] == 'ADULTOS'].iloc[0].value_a)
    IN1_01_CA_OTR_N = int(dfCa.loc[df['age_gender'] == 'OTRO'].iloc[0].value_a)

    dfDiversidad = osmosys.osmosys.getRefLgbtiDiscapacitadosValues(year=year, month=month, orgOsmosys=orgAcron,
                                                                   indicatorsIdsOmosys=indicatorIdsOsmosysServicios,
                                                                   cantonCode=cantonCode)

    IN1_01_RM_LGBT_N = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'LGBTI') & (
            dfDiversidad['country_of_origin'] == 'VENEZUELA')].iloc[0].value_a)

    IN1_01_CA_LGBT_N = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'LGBTI') & (
            dfDiversidad['country_of_origin'] == 'ECUADOR')].iloc[0].value_a)

    commentary = osmosys.osmosys.getCommentary(year=year, month=month, orgOsmosys=orgAcron,
                                               indicatorsIdsOmosys=indicatorIdsOsmosysServicios).iloc[
        0].value_a

    subform = model.modelAI.SubFormIN1_01(
        mes=month_number,
        colltmgkykvhxgij6=indicatorIdAI,
        rmrp='Si',
        poblacion_meta=poblacion_meta,
        poblacion_meta_freq='Primera vez',
        modalidad_impl='cmo12q1kykwwosjka',  ## Provisión de servicio
        mecanismos=None,
        transferencia=None,
        usd_transfer=None,
        IN1_01_RM_MM_N=IN1_01_RM_MM_N,
        IN1_01_RM_HH_N=IN1_01_RM_HH_N,
        IN1_01_RM_OTR_N=IN1_01_RM_OTR_N,
        IN1_01_RM_LGBT_N=IN1_01_RM_LGBT_N,
        IN1_01_CA_MM_N=IN1_01_CA_MM_N,
        IN1_01_CA_HH_N=IN1_01_CA_HH_N,
        IN1_01_CA_OTR_N=IN1_01_CA_OTR_N,
        IN1_01_CA_LGBT_N=IN1_01_CA_LGBT_N,
        IN1_01_CUAL=commentary
    )
    newId = generate_id()
    newIds.append(newId)
    record = model.modelAI.Record(formId=formIdAI, recordId=newId, parentRecordId=row['@id'], fields=subform)
    changesList.append(record)
## Los que son transferencias
for index, row in parentFormsIdsTransferencias.iterrows():
    # print(row)
    orgAcron = row.acronym
    cantonCode = row.canton_code
    df = osmosys.osmosys.getRefValues(year=year, month=month, orgOsmosys=orgAcron,
                                      indicatorsIdsOmosys=indicatorIdsOsmosysTranferencias, cantonCode=cantonCode)

    poblacion_meta = ["Refugiados/as y migrantes", "Comunidad de acogida"]

    IN1_01_RM_MM_R = int(df.loc[df['age_gender'] == 'ADULTAS'].iloc[0].value_a)
    IN1_01_RM_HH_R = int(df.loc[df['age_gender'] == 'ADULTOS'].iloc[0].value_a)
    IN1_01_RM_OTR_R = int(df.loc[df['age_gender'] == 'OTRO'].iloc[0].value_a)

    dfCa = osmosys.osmosys.getCAValues(year=year, month=month, orgOsmosys=orgAcron,
                                       indicatorsIdsOmosys=indicatorIdsOsmosysTranferencias, cantonCode=cantonCode)

    IN1_01_CA_MM_R = int(dfCa.loc[df['age_gender'] == 'ADULTAS'].iloc[0].value_a)
    IN1_01_CA_HH_R = int(dfCa.loc[df['age_gender'] == 'ADULTOS'].iloc[0].value_a)
    IN1_01_CA_OTR_R = int(dfCa.loc[df['age_gender'] == 'OTRO'].iloc[0].value_a)

    dfDiversidad = osmosys.osmosys.getRefLgbtiDiscapacitadosValues(year=year, month=month, orgOsmosys=orgAcron,
                                                                   indicatorsIdsOmosys=indicatorIdsOsmosysTranferencias,
                                                                   cantonCode=cantonCode)

    IN1_01_RM_LGBT_R = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'LGBTI') & (
            dfDiversidad['country_of_origin'] == 'VENEZUELA')].iloc[0].value_a)

    IN1_01_CA_LGBT_R = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'LGBTI') & (
            dfDiversidad['country_of_origin'] == 'ECUADOR')].iloc[0].value_a)

    commentary = osmosys.osmosys.getCommentary(year=year, month=month, orgOsmosys=orgAcron,
                                               indicatorsIdsOmosys=indicatorIdsOsmosysTranferencias).iloc[
        0].value_a

    usd_transfer = int(osmosys.osmosys.getCBIBudget(year=year, month=month, orgOsmosys=orgAcron,
                                                    indicatorsIdsOmosys=indicatorIdsOsmosysTranferencias,
                                                    cantonCode=cantonCode).iloc[0].budget)

    subform = model.modelAI.SubFormIN1_01(
        mes=month_number,
        colltmgkykvhxgij6=indicatorIdAI,
        rmrp='Si',
        poblacion_meta=poblacion_meta,
        poblacion_meta_freq='Primera vez',
        modalidad_impl='c3jf6gykykwwemuk7',  ## EFECTIVO
        mecanismos='c3de3pmkym0dbino', ## Entrega en ATM/Cajero automático
        transferencia='Incondicional',
        usd_transfer=usd_transfer,
        IN1_01_RM_MM_N=IN1_01_RM_MM_N,
        IN1_01_RM_HH_N=IN1_01_RM_HH_N,
        IN1_01_RM_OTR_N=IN1_01_RM_OTR_N,
        IN1_01_RM_LGBT_N=IN1_01_RM_LGBT_N,
        IN1_01_CA_MM_N=IN1_01_CA_MM_N,
        IN1_01_CA_HH_N=IN1_01_CA_HH_N,
        IN1_01_CA_OTR_N=IN1_01_CA_OTR_N,
        IN1_01_CA_LGBT_N=IN1_01_CA_LGBT_N,
        IN1_01_CUAL=commentary
    )
    newId = generate_id()
    newIds.append(newId)
    record = model.modelAI.Record(formId=formIdAI, recordId=newId, parentRecordId=row['@id'], fields=subform)
    changesList.append(record)
print('changes to charge: ' + str(len(changesList)))

changes to charge: 3


In [35]:
## creo respaldo
changes = model.modelAI.Changes(changesList)
finalJson = json.dumps(changes, default=model.modelAI.default)

In [36]:
#open text file
try:
    os.mkdir(month)
except Exception:
    pass
text_file = open(os.path.join(month, "data_" + indicatorCodeAI + '-' + str(indicatorIdsOsmosys) + ".json"), "w")
#write string to file
n = text_file.write(finalJson)

#close file
text_file.close()
print(" creado respaldo: " + text_file.name)
newIdsDict = {"newIds": newIds}
newIdsDf = pd.DataFrame(newIdsDict)
newIdsDf.to_csv(os.path.join(month, "new_ids_" + indicatorCodeAI + ".csv"))

 creado respaldo: ENERO\data_IN1_01-[265 267 270 276 278].json


In [37]:
## envio a AI
if (False):
    print(
        '--------------------------------------' + indicatorCodeAI + '-' + str(
            indicatorIdsOsmosys) + '------------------------------------------------------')
else:
    try:
        if len(changesList) > 0:
            print('se envia a AI')
            client = Client(token=osmosys.osmosys.getToken(), base_url='https://www.activityinfo.org/resources')
            client.post_resource(path='update', body=finalJson)
            print(
                '--------------------------------------' + indicatorCodeAI + '-' + str(
                    indicatorIdsOsmosys) + '------------------------------------------------------')
        else:
            print('nada que enviar a AI')
    except HTTPError as e:
        code = e.response.status_code
        print('error:')
        print(code)

        print(e)

se envia a AI
--------------------------------------IN1_01-[265 267 270 276 278]------------------------------------------------------
